In [1]:
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
import random

2023-06-04 20:56:58.321812: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 20:56:58.354740: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-04 20:56:58.355657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 20:56:59.152054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.__version__

'2.12.0'

In [3]:
DATASET_PATH = '/home/irizqy/ml_ws/bangkit-ws/data/bizz.it-sim_dataset'

In [4]:
im_path_arr = os.listdir(DATASET_PATH)
im_classes_arr = []

for file in im_path_arr:
    im_classes_arr.append(file.split('_')[0])

im_classes_arr = np.asarray(im_classes_arr)

In [5]:
classes = np.unique(im_classes_arr)

dict_keys = {val:key for key, val in enumerate(classes.flatten())}

In [6]:
grouped_im_path = [np.where(im_classes_arr == cls)[0] for cls in classes]

In [7]:

def make_pairs(ims_arr):
    im_pairs = []
    labels = []

    for file in ims_arr:
        file_cls = file.split('_')[0]
        cls = dict_keys[file_cls]

        current_im = cv.imread(os.path.join(DATASET_PATH, file))
        # current_im = cv.cvtColor(current_im, cv.COLOR_BGR2RGB)
        current_im = cv.cvtColor(current_im, cv.COLOR_BGR2GRAY)
        current_im = cv.resize(current_im, (150, 150))
        current_im = current_im / 255

        pos_idx = np.random.choice(grouped_im_path[cls], 1)[0]
        pos_pair_im = cv.imread(os.path.join(DATASET_PATH, im_path_arr[pos_idx]))
        # pos_pair_im = cv.cvtColor(pos_pair_im, cv.COLOR_BGR2RGB)
        pos_pair_im = cv.cvtColor(pos_pair_im, cv.COLOR_BGR2GRAY)
        pos_pair_im = cv.resize(pos_pair_im, (150, 150))
        pos_pair_im = pos_pair_im / 255

        im_pairs.append((current_im, pos_pair_im))
        labels.append(1)

        neg_idx = np.random.choice(np.where(im_classes_arr != file_cls)[0], 1)[0]
        neg_pair_im = cv.imread(os.path.join(DATASET_PATH, im_path_arr[neg_idx]))
        # neg_pair_im = cv.cvtColor(neg_pair_im, cv.COLOR_BGR2RGB)
        neg_pair_im = cv.cvtColor(neg_pair_im, cv.COLOR_BGR2GRAY)
        neg_pair_im = cv.resize(neg_pair_im, (150, 150))
        neg_pair_im = neg_pair_im / 255

        im_pairs.append((current_im, neg_pair_im))
        labels.append(0)

    return np.asarray(im_pairs), np.asarray(labels)

In [8]:
train_pairs, train_labels = make_pairs(im_path_arr)

In [9]:
def split_train_to_test(ims_arr, ims_label, percentage):
    arr_length = ims_arr.shape[0]
    num_of_data = int((percentage/100) * arr_length)
    im_pairs = []
    im_labels = []


    for i in range(num_of_data):
        rand_i = np.random.randint(arr_length - 1)
        im_pairs.append(ims_arr[rand_i])
        im_labels.append(ims_label[rand_i])
    
    return np.asarray(im_pairs), np.asarray(im_labels)

In [10]:
test_pairs, test_label = split_train_to_test(train_pairs, train_labels, 10)

In [ ]:
w = 20
h = 15
fig = plt.figure(figsize=(8, 8))
columns = 8
rows = 8
for index, i in enumerate(range(1, (columns*rows +1)//2)):
    img = train_pairs[index][0]
    fig.add_subplot(rows, columns, 2*i - 1) 
    if i % 2 == 1:
        plt.title('1')
    else:
        plt.title('0')
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    img = train_pairs[index][1]
    fig.add_subplot(rows, columns, 2*i)
    plt.imshow(img, cmap='gray')
    plt.axis('off')


In [ ]:
plt.imshow(np_ims[2][0], cmap='gray')

In [ ]:
IMG_SHAPE = (150, 150, 1)
BATCH_SIZE = 8
EPOCHS = 100

In [ ]:
class SiameseModel:

    def __init__(self, input_shape, embedding_dim=300):
        self.input_shape = input_shape
        self.embedding_dim = embedding_dim

    def _build(self):
        inputs = tf.keras.layers.Input(self.input_shape)
        
        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(inputs)
        x = tf.keras.layers.MaxPooling2D()(x)
        x = tf.keras.layers.Dropout(.1)(x)

        x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
        x = tf.keras.layers.MaxPooling2D()(x)

        # x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
        # x = tf.keras.layers.MaxPooling2D()(x)

        # x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(x)
        # x = tf.keras.layers.MaxPooling2D()(x)
        # x = tf.keras.layers.Dropout(.3)(x)

        # pooled_output = tf.keras.layers.GlobalAveragePooling2D()(x)
        pooled_output = tf.keras.layers.Flatten()(x)
        outputs = tf.keras.layers.Dense(self.embedding_dim)(pooled_output)

        model = tf.keras.Model(inputs, outputs)

        return model

In [ ]:
sm = SiameseModel(IMG_SHAPE)

featureExtractor = sm._build()
featureExtractor.summary()

# base_cnn =  tf.keras.applications.resnet.ResNet50(
#     weights=None, input_shape=IMG_SHAPE, include_top=False
# )

# cnn_model = base_cnn.get_layer('conv5_block3_2_conv')

# flatten = tf.keras.layers.Flatten()(cnn_model.output)
# dense_1 = tf.keras.layers.Dense(units=256, activation='relu')(flatten)
# dense_2 = tf.keras.layers.Dense(units=128, activation='relu')(dense_1)

# featureExtractor = tf.keras.Model(base_cnn.input, dense_2)

# for layer in featureExtractor.layers[:-25]:
#     layer.trainable = False

In [ ]:
featureExtractor.summary()

In [ ]:
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(feats_A, feats_B) = vectors
	# compute the sum of squared distances between the vectors
	sum_squared = K.sum(K.square(feats_A - feats_B), axis=1,
		keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sum_squared, K.epsilon()))

In [ ]:
def cosine_similarity(vectors):
    (featsA, featsB) = vectors
    sum_product = K.sum(featsA*featsB)
    sum_squared_featsA = K.sqrt(K.sum(featsA**2, keepdims=1, axis=1))
    sum_squared_featsB = K.sqrt(K.sum(featsB**2, keepdims=1, axis=1))
    sum_mul_feats = sum_squared_featsA * sum_squared_featsB

    return sum_product / sum_mul_feats

In [ ]:
def plot_training(H):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	# plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	# plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")

In [ ]:
# configure the siamese network
print("[INFO] building siamese network...")
imgA = tf.keras.layers.Input(shape=IMG_SHAPE)
imgB = tf.keras.layers.Input(shape=IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

In [ ]:
# finally, construct the siamese network
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[imgA, imgB], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
tf.keras.backend.clear_session()
# compile the model
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam",
	metrics=["accuracy"])
# train the model
print("[INFO] training model...")
history = model.fit(
	[np_ims[:, 0], np_ims[:, 1]], np_labels[:],
	# validation_data=([np_ims[251:, 0], np_ims[251:, 1]], np_labels[251:]),
	batch_size=BATCH_SIZE, 
	epochs=EPOCHS)

In [ ]:
plot_training(history)

In [ ]:
model.save('/home/irizqy/ml_ws/bangkit-ws/src/logo-detector/im_similar')

In [ ]:
from PIL import Image, ImageOps

preds = []

test_data = os.listdir('/home/irizqy/ml_ws/bangkit-ws/data/bizz.it-sim_dataset')

im1 = Image.open('/home/irizqy/ml_ws/bangkit-ws/src/ops/cropped-logo.jpg').resize((150, 150))
im1 = ImageOps.grayscale(im1)
im1.show()
im1 = np.asarray(im1)
im1 = im1 / 255

image1 = np.expand_dims(im1, 0)

im2 = Image.open('/home/irizqy/ml_ws/bangkit-ws/data/bizz.it-sim_dataset/mcd_85.jpg').resize((150, 150))
# im2 = Image.open('/home/irizqy/ml_ws/bangkit-ws/data/LogoDet-3K/Food/mcdonalds/3.jpg').resize((150, 150))
im2 = ImageOps.grayscale(im2)
im2.show()
im2 = np.asarray(im2)
im2 = im2 / 255

image2 = np.expand_dims(im2, 0)
model.predict((image1, image2))[0][0]

# for file in test_data:
#     im2 = Image.open(os.path.join('/home/irizqy/ml_ws/bangkit-ws/data/bizz.it-sim_dataset', file)).resize((150, 150))
#     im2 = ImageOps.grayscale(im2)
#     im2 = np.asarray(im2)
#     im2 = im2 / 255

#     image2 = np.expand_dims(im2, 0)
#     pred = model.predict((image1, image2))[0][0]
#     preds.append(pred)

# preds = np.asarray(preds)

In [ ]:
max_idx = np.argmax(preds)
print(max_idx)

In [ ]:
test_data[max_idx]